# Считаем нашы датафреймы

In [130]:
import pandas as pd
import numpy as np

In [131]:
ls -lh ../input/

total 71M
-rw-r--r-- 1 nobody nogroup 166K Apr 29 17:48 sample_submission.csv
-rw-r--r-- 1 nobody nogroup  19M Apr 29 17:48 test.csv
-rw-r--r-- 1 nobody nogroup 2.3M Apr 29 17:48 test.csv.zip
-rw-r--r-- 1 nobody nogroup  43M Apr 29 17:48 train.csv
-rw-r--r-- 1 nobody nogroup 7.6M Apr 29 17:48 train.csv.zip


In [205]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
test_df_id = test_df.Id.values

# **Уберем ненужные фичужки и сгенерим бинарные из колонки 'features'**

In [133]:
def get_features(feature_list):
    return [feature for feature in feature_list.split('\'') if feature !=', ' and feature != ']' and feature != '[' and feature != '[]']
    

from collections import Counter
drop_columns = ['building_id', 'created', 'description', 'display_address', 'latitude', 'longitude', 'manager_id', 'photos', 'street_address', 'listing_id']
train_df.drop(drop_columns, axis=1, inplace=True)
test_df.drop(drop_columns, axis=1, inplace=True)
train_df['features'] = train_df['features'].apply(get_features)
test_df['features'] = test_df['features'].apply(get_features)

In [134]:
unique_features = []
for feature_list in train_df['features']:
    for feature in feature_list:
        unique_features.append(feature)
        
new_features = [new_feature[0] for new_feature in Counter(unique_features).most_common()[:47]]
for new_feature in new_features:
    train_df[new_feature] = 0

In [135]:
def find_feature(features, feature):
    if feature in features:
        return 1
    return 0

def fill_new_features(df):
    for new_feature in new_features:
        df[new_feature] = df.apply(
            lambda row: find_feature(row['features'], new_feature), axis=1)
        
fill_new_features(train_df)
fill_new_features(test_df)

In [136]:
def add_2_more_features(df):
    df['bedroom price'] = df['price'] / (df['bedrooms'] + 0.1)
    df['bathroom price'] = df['price'] / (df['bathrooms'] + 0.1)

add_2_more_features(train_df)
add_2_more_features(test_df)
train_df.drop('features', axis=1, inplace=True)
test_df.drop('features', axis=1, inplace=True)

In [137]:
train_df.describe()

,Id,bathrooms,bedrooms,price,Elevator,Hardwood Floors,Cats Allowed,Dogs Allowed,Doorman,Dishwasher,No Fee,Laundry in Building,Fitness Center,Pre-War,Laundry in Unit,Roof Deck,Outdoor Space,Dining Room,High Speed Internet,Balcony,Swimming Pool,Laundry In Building,New Construction,Terrace,Exclusive,Loft,Garden/Patio,Wheelchair Access,Common Outdoor Space,HARDWOOD,SIMPLEX,Fireplace,prewar,LOWRISE,Garage,Laundry Room,Reduced Fee,Laundry In Unit,Furnished,Multi-Level,Prewar,Private Outdoor Space,PublicOutdoor,Parking Space,Roof-deck,dishwasher,elevator,High Ceilings,Renovated,Pool,LAUNDRY,bedroom price,bathroom price
count,34546.000000,34546.000000,34546.000000,3.454600e+04,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,34546.000000,3.454600e+04,3.454600e+04
mean,61873.351618,1.211182,1.540815,3.888823e+03,0.528484,0.478203,0.476640,0.446448,0.425317,0.414983,0.364644,0.332368,0.268048,0.184537,0.178602,0.132924,0.106206,0.105251,0.087246,0.061107,0.055318,0.053002,0.052162,0.046431,0.043565,0.042726,0.038876,0.027992,0.026457,0.018787,0.018671,0.018671,0.017455,0.015950,0.015197,0.014792,0.014387,0.014213,0.014010,0.013026,0.010218,0.009987,0.008771,0.008481,0.008250,0.007613,0.007497,0.007179,0.006426,0.005587,0.005529,6.682276e+03,3.166193e+03
std,35718.160364,0.496217,1.116735,2.630662e+04,0.499195,0.499532,0.499461,0.497131,0.494398,0.492726,0.481337,0.471069,0.442949,0.387927,0.383025,0.339498,0.308106,0.306881,0.282199,0.239530,0.228602,0.224040,0.222357,0.210419,0.204128,0.202241,0.193301,0.164951,0.160494,0.135772,0.135361,0.135361,0.130961,0.125283,0.122338,0.120721,0.119080,0.118369,0.117535,0.113388,0.100569,0.099435,0.093243,0.091705,0.090455,0.086921,0.086263,0.084425,0.079907,0.074537,0.074152,1.740998e+04,2.419965e+04
min,6.000000,0.000000,0.000000,4.500000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.009804e+02,4.090909e+01
25%,30839.250000,1.000000,1.000000,2.500000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.523810e+03,2.142857e+03
50%,61822.500000,1.000000,1.000000,3.150000e+03,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.268182e+03,2.631818e+03
75%,92705.500000,1.000000,2.000000,4.100000e+03,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

# Выносим отдельно переменную Target

In [138]:
X_train = train_df.drop(['TARGET'], axis=1)
X_test = test_df

y_train = train_df.loc[:, 'TARGET']

In [139]:
def label_to_num(label):
    if label == 'low':
        return 0
    if label == 'medium':
        return 1
    return 2

y_train = y_train.apply(label_to_num).values

# Фитаем регрессию

In [140]:
from sklearn.linear_model import LogisticRegression

In [141]:
lg = LogisticRegression(multi_class='ovr', solver='lbfgs',penalty='l2',
                        class_weight={0:0.33, 2:2.9, 1:1.})

In [145]:
from sklearn.model_selection import cross_val_score

# Результат для исходных данных
cross_val_score(lg, X_train, y_train, scoring='accuracy', cv=5).mean()

0.5443749352485601

# **Заюзаем SFS для лучших результатов**

In [204]:
from mlxtend.feature_selection import SequentialFeatureSelector

selector = SequentialFeatureSelector(LogisticRegression(multi_class='ovr', solver='lbfgs',penalty='l2',
                        class_weight={0:0.33, 2:2.9, 1:1.}), scoring='accuracy',
                                     verbose=2, k_features=45, forward=False, n_jobs=-1)

selector.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  52 out of  52 | elapsed:  5.4min finished

[2022-04-29 21:24:51] Features: 51/45 -- score: 0.5755515234311511[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  51 out of  51 | elapsed:  5.4min finished

[2022-04-29 21:30:18] Features: 50/45 -- score: 0.5787647445596008[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  4.8min finished

[2022-04-29 21:35:07] Features: 49/45 -- score: 0.5833095622170685[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  49 ou

SequentialFeatureSelector(clone_estimator=True, cv=5,
             estimator=LogisticRegression(C=1.0, class_weight={0: 0.33, 2: 2.9, 1: 1.0}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=None, penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False),
             floating=False, forward=False, k_features=45, n_jobs=-1,
             pre_dispatch='2*n_jobs', scoring='accuracy', verbose=2)

In [207]:
train_features = selector.k_feature_names_

45

In [217]:
cross_val_score(LogisticRegression(multi_class='ovr', solver='lbfgs',penalty='l2',
                        class_weight={0:0.33, 2:2.9, 1:1.}, max_iter=1000), X_train[list(train_features)], y_train, scoring='accuracy', cv=5).mean()

0.5885776588408209

**Вот такой результат дал SFS. Теперь проверим, что он не ограничил фичи до того, чтобы просто выдать больший класс *low***.

In [244]:
indexes = []
for cnt, label in enumerate(y_train):
    if label > 0 or cnt < 1000:
        indexes.append(cnt)
        
cross_val_score(LogisticRegression(multi_class='ovr', solver='lbfgs',penalty='l2',
                        class_weight={0:0.33, 2:2.9, 1:1.}, max_iter=1000), X_train[list(train_features)].iloc[indexes], y_train[indexes], scoring='accuracy', cv=5).mean()

0.5909867931988928

In [ ]:
def num_to_label(num):
    if num == 0:
        return 'low'
    if num == 1:
        return 'medium'
    return 'high'

lg = LogisticRegression(multi_class='ovr', solver='lbfgs',penalty='l2',
                        class_weight={0:0.33, 2:2.9, 1:1.})
lg.fit(X_train[list(train_features)], y_train)
y_pred = lg.predict(X_test[list(train_features)])
y_pred = np.stack(np.vectorize(num_to_label)(y_pred))

In [245]:
from collections import Counter
Counter(y_pred).most_common()

[('low', 8803), ('high', 3758), ('medium', 2245)]

# Делаем сабмит

In [248]:
submit = pd.DataFrame.from_dict({'Id':test_df_id, 'TARGET': y_pred})
submit.to_csv("submit.csv", index=False)

In [254]:
correct_labels = submit[submit['Id'] == submit['TARGET']].shape[0]

In [257]:
print(f'Точность на тесте: {correct_labels / test_df.shape[0] * 100} %')

Точность на тесте: 57.55774685938133 %
